In [1]:
from game import Game
from ai import SplendorAI
from player import get_phase_parameters
from constants import *
from datetime import datetime
from player import get_phase_parameters
import sys
from collections import defaultdict



/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# simply used to initialize players
base_game = Game(id=0, n_players=4)

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3048142096666187465
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10037365965
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10770554953093589006
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [4]:
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [5]:
# temperature will decrease slightly quicker over time
def calculate_temperature(round):
    return 2/(1+round/1.2) * 1/(1+round/8) * 1/(1+round/20)

In [6]:
players = base_game.players
game_data = defaultdict(list)



In [19]:
set_durations = {}

[0.3, 0.25, 0.24166666666666667, 0.20833333333333334]

# Test multiple runs of games

In [20]:
n_rounds = 20
n_sets_per_round = 2
n_simulations_per_set = 120

start_time = datetime.now()
for i in range(n_rounds):
    print('ON ROUND', i)
    for j in range(n_sets_per_round):
        set_start_time = datetime.now()
        for k in range(n_simulations_per_set):
            # if (i==0) and (j==0):
            #    # soft restart
            #    break
            new_game = Game(id=i*200 + j*100+k, players=players)
            stalemate = new_game.run()  
            # stalemates should be extraordinarily rare and pretty much nonexistent
            if stalemate: 
                very_long_game = new_game
                print('teaching ai not to stalemate')
                for player in players:
                    player.transfer_history_to_ai()
                    player.ai.train_models(verbose=0,n_epochs=4)
            sys.stdout.write('.')
            sys.stdout.flush()
            # record historic game data for each set
            game_data[(i,j)].append(new_game.copy_plain_data_for_self_and_players())
        set_stop_time = datetime.now()
        duration = (set_start_time-set_stop_time).seconds
        for player in players:
            player.transfer_history_to_ai()
            player.ai.train_models(verbose=0, n_epochs=12)
        set_durations[(i,j,k)] = duration
        print('/')
        sys.stdout.flush()
        avg_game_length = np.mean([x['game']['turn'] for x in game_data[(i,j)]])
        print('R/S %d/%d AVERAGE GAME LENGTH: ' % (i, j), avg_game_length)
        avg_cards_purchased = np.mean([
            [
                pdata['n_cards'] 
                for pdata in x['players'].values()
            ] 
            for x in game_data[(i,j)]
        ]
        )
        print('R/S %d/%d AVERAGE CARDS PURCHASED: ' % (i, j), avg_cards_purchased)
        # calculate win rates
        player_data = [x['players'] for x in game_data[(i,j)]]
        win_values = [[player_data[idx][pid]['win'] for idx in range(120)] for pid in range(4)]
        win_rates = [np.mean(v) for v in win_values]
        print('WIN RATES: ', str(win_rates))
                             
    for player in players:
        player.reset(reset_extended_history=True)
    phase = min(5, i // 4 + 1)
    temperature = calculate_temperature(i)
    
    for p_i, player in enumerate(players):
        player.temperature = temperature
        player.decision_weighting = get_phase_parameters(phase)
        model_name = 'run5t2_player_%s_round_' % str(p_i)
        player.ai.save_models(model_name, index=i)
    
    
# stop_time = datetime.now()
for time in [start_time, stop_time]:
    print(time.strftime('%x %X'))

ON ROUND 0
training win model
training Q1 model
training Q3 model
training Q5 model
training win model
training Q1 model
training Q3 model
training Q5 model
training win model
training Q1 model
training Q3 model
training Q5 model
training win model
training Q1 model
training Q3 model
training Q5 model
/
R/S 0/0 AVERAGE GAME LENGTH:  147.26666666666668
R/S 0/0 AVERAGE CARDS PURCHASED:  13.604166666666666
WIN RATES:  [0.3, 0.25, 0.24166666666666667, 0.20833333333333334]
........................................................................................................................training win model
training Q1 model
training Q3 model
training Q5 model
training win model
training Q1 model
training Q3 model
training Q5 model
training win model
training Q1 model
training Q3 model
training Q5 model
training win model
training Q1 model
training Q3 model
training Q5 model
/
R/S 0/1 AVERAGE GAME LENGTH:  128.2
R/S 0/1 AVERAGE CARDS PURCHASED:  13.454166666666667
WIN RATES:  [0.266666666

saving run5t2_player_0_round__Q3_4.h5
saving run5t2_player_0_round__Q5_4.h5
saving run5t2_player_1_round__win_4.h5
saving run5t2_player_1_round__Q1_4.h5
saving run5t2_player_1_round__Q3_4.h5
saving run5t2_player_1_round__Q5_4.h5
saving run5t2_player_2_round__win_4.h5
saving run5t2_player_2_round__Q1_4.h5
saving run5t2_player_2_round__Q3_4.h5
saving run5t2_player_2_round__Q5_4.h5
saving run5t2_player_3_round__win_4.h5
saving run5t2_player_3_round__Q1_4.h5
saving run5t2_player_3_round__Q3_4.h5
saving run5t2_player_3_round__Q5_4.h5
ON ROUND 5
........................................................................................................................training win model
training Q1 model
training Q3 model
training Q5 model
training win model
training Q1 model
training Q3 model
training Q5 model
training win model
training Q1 model
training Q3 model
training Q5 model
training win model
training Q1 model
training Q3 model
training Q5 model
/
R/S 5/0 AVERAGE GAME LENGTH:  120.733333

/
R/S 9/0 AVERAGE GAME LENGTH:  118.4
R/S 9/0 AVERAGE CARDS PURCHASED:  14.022916666666667
WIN RATES:  [0.23333333333333334, 0.39166666666666666, 0.25, 0.125]
........................................................................................................................training win model
training Q1 model
training Q3 model
training Q5 model
training win model
training Q1 model
training Q3 model
training Q5 model
training win model
training Q1 model
training Q3 model
training Q5 model
training win model
training Q1 model
training Q3 model
training Q5 model
/
R/S 9/1 AVERAGE GAME LENGTH:  119.1
R/S 9/1 AVERAGE CARDS PURCHASED:  14.402083333333334
WIN RATES:  [0.23333333333333334, 0.325, 0.20833333333333334, 0.24166666666666667]
saving run5t2_player_0_round__win_9.h5
saving run5t2_player_0_round__Q1_9.h5
saving run5t2_player_0_round__Q3_9.h5
saving run5t2_player_0_round__Q5_9.h5
saving run5t2_player_1_round__win_9.h5
saving run5t2_player_1_round__Q1_9.h5
saving run5t2_player_1_ro

saving run5t2_player_2_round__Q1_13.h5
saving run5t2_player_2_round__Q3_13.h5
saving run5t2_player_2_round__Q5_13.h5
saving run5t2_player_3_round__win_13.h5
saving run5t2_player_3_round__Q1_13.h5
saving run5t2_player_3_round__Q3_13.h5
saving run5t2_player_3_round__Q5_13.h5
ON ROUND 14
........................................................................................................................training win model
training Q1 model
training Q3 model
training Q5 model
training win model
training Q1 model
training Q3 model
training Q5 model
training win model
training Q1 model
training Q3 model
training Q5 model
training win model
training Q1 model
training Q3 model
training Q5 model
/
R/S 14/0 AVERAGE GAME LENGTH:  116.66666666666667
R/S 14/0 AVERAGE CARDS PURCHASED:  14.191666666666666
WIN RATES:  [0.09166666666666666, 0.35, 0.36666666666666664, 0.19166666666666668]
..................................................................................................................

........................................................................................................................training win model
training Q1 model
training Q3 model
training Q5 model
training win model
training Q1 model
training Q3 model
training Q5 model
training win model
training Q1 model
training Q3 model
training Q5 model
training win model
training Q1 model
training Q3 model
training Q5 model
/
R/S 18/1 AVERAGE GAME LENGTH:  116.23333333333333
R/S 18/1 AVERAGE CARDS PURCHASED:  14.204166666666667
WIN RATES:  [0.16666666666666666, 0.325, 0.3333333333333333, 0.175]
saving run5t2_player_0_round__win_18.h5
saving run5t2_player_0_round__Q1_18.h5
saving run5t2_player_0_round__Q3_18.h5
saving run5t2_player_0_round__Q5_18.h5
saving run5t2_player_1_round__win_18.h5
saving run5t2_player_1_round__Q1_18.h5
saving run5t2_player_1_round__Q3_18.h5
saving run5t2_player_1_round__Q5_18.h5
saving run5t2_player_2_round__win_18.h5
saving run5t2_player_2_round__Q1_18.h5
saving run5t2_player_

NameError: name 'stop_time' is not defined

In [ ]:
from collections import Mapping, Container 
from sys import getsizeof

def deep_getsizeof(o, ids): 

    d = deep_getsizeof
    if id(o) in ids:
        return 0

    r = getsizeof(o)
    ids.add(id(o))

    if isinstance(o, str) or isinstance(0, bytes):
        return r
    
    if isinstance(o, np.ndarray):
        return r

    if isinstance(o, Mapping):
        return r + sum(d(k, ids) + d(v, ids) for k, v in o.items())

    if isinstance(o, Container):
        return r + sum(d(x, ids) for x in o)

    return r

deep_getsizeof(players[0].extended_serialized_action_history[0], set())

# for k in list(locals().keys()):
#    v = locals()[k]
#    size = deep_getsizeof(v, set())
#    if size > 100000:
#        print(k, ':', size)

In [21]:
import pickle

with open('run_5_game_data.dat', 'wb') as f:
    pickle.dump(game_data, f)
    
with open('run_5_durations.dat', 'wb') as f:
    pickle.dump(set_durations, f)
    


In [ ]:
new_game = Game(id=i*200 + j*100+1.1, players=players)

In [ ]:
for k in dir(players[2]):
    v = getattr(players[2], k)
    if isinstance(v, (list, dict)):
        print(k, len(v))

In [ ]:
len(players[0].ai.extended_serialized_history)

In [ ]:
new_game.available_tier_3_cards

In [ ]:
new_game.serialize()['available_cards']

In [ ]:
[sum([card['points'] for card in player.owned_cards]) for player in players]

In [ ]:
[sum([card['points'] for card in player.reserved_cards if card['tier']==3]) for player in players]

In [ ]:
[player.points for player in players]

In [ ]:
[player.gems  for player in players]

In [ ]:
new_game.gems

In [ ]:
[player.active_turn for player in players]

In [ ]:
set_durations

In [ ]:
players[2].simulate_purchasing_options()['score']

In [ ]:
players[2].simulate_gem_taking_options()['score']

In [ ]:
# looks like i had an oversight in the decisionmaking code when temperatures were higher
for z in range(10000):
    if players[2].make_choice([13.436, -1e100, 11.29427]) == 1:
        print('uh oh')
        print(z)
        break

In [ ]:
arr = np.asarray([13.436, -1e100, 11.29427])
arr2 = np.exp(arr/players[2].temperature)
print(arr2)
print(arr2/np.sum(arr2)==0.)


In [ ]:
for k, v in game_data.items():
    print(len(v))

# NOTES

It appears that the code broke in phase 5, where winning is half the weight of the overall Q-score. The game broke in the last turn of the 4th player due to insufficient input size. Player 2 was about to win with 17 points (2 objectives!) and player 4 was not doing as well.